In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error, mean_squared_error
from collections import defaultdict
import math
from tqdm import tqdm
import seaborn as sns
import re
import os
import transformers
import json
import torch
#import isodate


C:\Users\lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load CSVs
recipes = pd.read_csv("recipes.csv", dtype={'CookTime': str, 'PrepTime': str, 'TotalTime': str, 'RecipeId': int})
reviews = pd.read_csv("reviews.csv")

print(recipes.shape, reviews.shape)
recipes.head()



(522517, 28) (1401982, 8)


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [3]:
print(recipes.shape, reviews.shape)
reviews.head()

(522517, 28) (1401982, 8)


,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z


In [4]:
# Drop duplicate recipes/reviews
recipes = recipes.drop_duplicates(subset="RecipeId")
reviews = reviews.drop_duplicates(subset="ReviewId")

nutritional_cols = [
    "Calories","FatContent","SaturatedFatContent","CholesterolContent",
    "SodiumContent","CarbohydrateContent","FiberContent","SugarContent","ProteinContent"
]
for col in nutritional_cols:
    recipes[col] = pd.to_numeric(recipes[col], errors="coerce")


In [5]:
def get_top_n(predictions, n=10): # Obtenida de https://surprise.readthedocs.io/en/stable/FAQ.html
    """Devuelve el top-N para cada usuario a partir de un conjunto de predicciones."""

    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Ordenar predicciones por score
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [6]:
def precision_recall_at_k(recs, relevant, k=10):
    recs_k = recs[:k]
    hits = len(set(recs_k) & set(relevant))
    precision = hits / k if k else 0
    recall = hits / len(relevant) if relevant else 0
    return precision, recall

def ndcg_at_k(recs, relevant, k=10):
    recs_k = recs[:k]
    dcg = sum(1 / math.log2(i+2) for i, iid in enumerate(recs_k) if iid in relevant)
    idcg = sum(1 / math.log2(i+2) for i in range(min(len(relevant), k)))
    return dcg / idcg if idcg > 0 else 0


In [7]:
def nutri_score(row):
    # Negative components
    neg = 0
    if not pd.isna(row["Calories"]): neg += min(10, row["Calories"] / 335)  # ~335 kJ = 80 kcal per point
    if not pd.isna(row["SugarContent"]): neg += min(10, row["SugarContent"] / 4.5)
    if not pd.isna(row["SaturatedFatContent"]): neg += min(10, row["SaturatedFatContent"] / 1)
    if not pd.isna(row["SodiumContent"]): neg += min(10, row["SodiumContent"] / 90)

    # Positive components
    pos = 0
    if not pd.isna(row["FiberContent"]): pos += min(5, row["FiberContent"] / 0.9)
    if not pd.isna(row["ProteinContent"]): pos += min(5, row["ProteinContent"] / 1.6)

    score = neg - pos
    if score <= -1: return "A"
    elif score <= 2: return "B"
    elif score <= 10: return "C"
    elif score <= 18: return "D"
    else: return "E"

recipes["NutriScore"] = recipes.apply(nutri_score, axis=1)
print(recipes[["RecipeId", "NutriScore"]].head(20))

    RecipeId NutriScore
0         38          C
1         39          D
2         40          D
3         41          D
4         42          C
5         43          D
6         44          D
7         45          C
8         46          B
9         47          B
10        48          E
11        49          D
12        50          B
13        51          B
14        52          B
15        53          C
16        54          D
17        55          C
18        56          E
19        57          A


In [8]:
def get_top_n(predictions, n=10):
    """Devuelve las N-mejores recomendaciones para cada usuario de un set de predicción.

    Args:
        predictions(lista de objetos Prediction): La lista de predicción obtenida del método test.
        n(int): El número de recomendaciónes por usuario

    Returns:
    Un diccionario donde las llaves son ids de usuario y los valores son listas de tuplas:
        [(item id, rating estimation), ...] de tamaño n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [9]:
def precision_recall_at_k(top_n, test_user_items, k=10):
    precisions = []
    recalls = []
    for uid, recs in top_n.items():
        if uid not in test_user_items:
            continue
        relevant = set(test_user_items[uid])
        recommended = set([iid for iid, _ in recs[:k]])
        precisions.append(len(recommended & relevant) / k)
        recalls.append(len(recommended & relevant) / len(relevant))
    return np.mean(precisions), np.mean(recalls)

def ndcg_at_k(top_n, test_user_items, k=10):
    ndcgs = []
    for uid, recs in top_n.items():
        if uid not in test_user_items:
            continue
        relevant = set(test_user_items[uid])
        dcg = 0
        for i, (iid, _) in enumerate(recs[:k]):
            if iid in relevant:
                dcg += 1 / math.log2(i + 2) 
        idcg = sum(1 / math.log2(i + 2) for i in range(min(k, len(relevant))))
        ndcgs.append(dcg / idcg if idcg > 0 else 0)
    return np.mean(ndcgs)

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    offload_folder="offload",  # cache folder for CPU layers
    max_memory={
        0: "6GiB",    # GPU
        "cpu": "16GiB"
    }
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)


In [11]:
for name, _ in model.named_modules():
    if "attn" in name or "proj" in name:
        print(name)


model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.mlp.gate_proj
model.layers.2.mlp.up_proj
model.layers.2.mlp.down_proj
model.layers.3.self_attn
model.layers.3.self_attn.q_proj
model.layers.3.self_attn.k_proj
model.layers.3.self_attn.v_proj
model.layers.3.self_attn.o_proj
model.layers.3.mlp.gate_proj
model.layers.3.mlp.up_proj
model.layers.3.mlp.down_proj
model.layers.4.self_attn
model.layers.4.self_att

In [12]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [14]:
# Build a dictionary for O(1) lookup of recipe names
recipe_names = dict(zip(recipes["RecipeId"], recipes["Name"]))

# Build user histories fast
user_histories = defaultdict(list)
for _, row in reviews.iterrows():
    rid = row["RecipeId"]
    if rid in recipe_names:
        user_histories[row["AuthorId"]].append((recipe_names[rid], row["Rating"]))

# Convert defaultdict to regular dict before saving
user_histories = dict(user_histories)

# Save to disk (JSON format)
with open("user_histories.json", "w", encoding="utf-8") as f:
    json.dump(user_histories, f, ensure_ascii=False, indent=2)


In [ ]:
# Load user histories if already saved
with open("user_histories.json", "r", encoding="utf-8") as f:
    user_histories = json.load(f)

In [14]:
import json, random

samples = []
for _, row in reviews.sample(10000, random_state=42).iterrows():  # limit for now
    uid, rid, rating = row["AuthorId"], row["RecipeId"], row["Rating"]
    recipe = recipes.loc[recipes["RecipeId"] == rid]
    if recipe.empty:
        continue

    name = recipe.iloc[0]["Name"]
    nutri = recipe.iloc[0]["NutriScore"]
    history = user_histories[uid][-5:] if uid in user_histories else []
    history_text = "\n".join([f"- {rname} ({rscore}/5)" for rname, rscore in history]) or "No previous ratings."

    prompt = (
        "A user has rated several recipes before. Based on their preferences "
        "and the healthiness of the recipe (NutriScore A–E), predict their rating (1–5).\n\n"
        f"User history:\n{history_text}\n\n"
        f"Target recipe: {name} (NutriScore {nutri})\n"
        "Answer only with a single number between 1 and 5 inclusive."
    )

    response = str(int(round(rating)))
    samples.append({"prompt": prompt, "response": response})



In [17]:
with open("train.jsonl", "w", encoding="utf-8") as f:
    for s in samples[:4000]:
        f.write(json.dumps(s) + "\n")

with open("val.jsonl", "w", encoding="utf-8") as f:
    for s in samples[4000:]:
        f.write(json.dumps(s) + "\n")


In [15]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={
    "train": "train.jsonl",
    "validation": "val.jsonl"
})


In [19]:
def reward(row):
    # Normalize rating (1–5 → 0–1)
    rating_score = (row["Rating"] - 1) / 4

    # Convert NutriScore (A–E) → health score (A=1, E=0)
    nutri_map = {"A": 1.0, "B": 0.8, "C": 0.6, "D": 0.3, "E": 0.0}
    health_score = nutri_map.get(row["NutriScore"], 0.5)

    # Weighted combination (you can tune these weights)
    return 0.7 * rating_score + 0.3 * health_score

In [16]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={"train": "train.jsonl", "validation": "val.jsonl"})

def format_dataset(example):
    text = f"{example['prompt']}\n\n{example['response']}"
    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

train_dataset = dataset["train"].map(format_dataset)
val_dataset = dataset["validation"].map(format_dataset)


Map: 100%|██████████| 5999/5999 [00:01<00:00, 3015.49 examples/s]


In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./tinyllama-finetuned2",
    per_device_train_batch_size=1,     
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    eval_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()


Step,Training Loss
10,15.741900
20,9.857600
30,3.751200
40,0.894500
50,0.477200
60,0.379200
70,0.269200
80,0.198900
90,0.136900
100,0.092000


TrainOutput(global_step=3000, training_loss=0.15761584680279095, metrics={'train_runtime': 3675.9153, 'train_samples_per_second': 3.264, 'train_steps_per_second': 0.816, 'total_flos': 3.8177788133376e+16, 'train_loss': 0.15761584680279095, 'epoch': 3.0})

In [18]:
model.save_pretrained("./tinyllama-finetuned2")
tokenizer.save_pretrained("./tinyllama-finetuned2")

from peft import PeftModel
from transformers import AutoModelForCausalLM

In [19]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(base_model, "./tinyllama-finetuned2")
model = model.merge_and_unload()

model.save_pretrained("./tinyllama-merged2")
tokenizer.save_pretrained("./tinyllama-merged2")


('./tinyllama-merged2\\tokenizer_config.json',
 './tinyllama-merged2\\special_tokens_map.json',
 './tinyllama-merged2\\chat_template.jinja',
 './tinyllama-merged2\\tokenizer.json')

In [20]:
import torch
import re
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer

class LLMPredictor:
    def __init__(self, recipes, user_histories, batch_size=10, model_path="./tinyllama-merged"):
        self.recipes = recipes
        self.user_histories = user_histories
        self.batch_size = batch_size

        print(f"Loading model from {model_path}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.model.eval()
        print("Model loaded and ready.")

    def _run_local(self, prompt):
        """Run local inference using the fine-tuned HF model."""
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

        with torch.no_grad():
            output = self.model.generate(
                **inputs,
                max_new_tokens=50,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                repetition_penalty=1.2
            )

        text = self.tokenizer.decode(output[0], skip_special_tokens=True)
        return text

    def predict_batch(self, pairs):
        prompt = (
            "You are predicting how much each user would rate a recipe. "
            "Output only a single whole number between 1 and 5 (inclusive) for each user, one per line. "
            "Do not use decimals or fractions. Consider the WHO Health Score into the recommendation, "
            "giving preference to healthier recipes.\n\n"
            "Respond ONLY with one rating per line in the same order.\n\n"
        )

        valid_pairs = []
        for i, (uid, iid, true_r) in enumerate(pairs, 1):
            recipe_row = self.recipes.loc[self.recipes["RecipeId"] == iid]
            if recipe_row.empty:
                continue

            recipe = recipe_row.iloc[0]
            history = self.user_histories.get(uid, [])
            history_snippet = "\n".join(
                [f"- Recipe: {name}, Rating: {r}" for name, r in history[-5:]]
            ) if history else "No past ratings available."

            prompt += (
                f"{i}. User {uid}:\n"
                f"Past ratings:\n{history_snippet}\n"
                f"Predict rating for this recipe:\n"
                f"Name: {recipe['Name']}, Health score: {recipe['NutriScore']}\n\n"
            )
            valid_pairs.append((uid, iid, true_r))

        if not valid_pairs:
            return []

        try:
            text = self._run_local(prompt).strip()
            numbers = re.findall(r"\d+", text)
            ratings = [min(max(int(n), 1), 5) for n in numbers]

            # Fill missing predictions with random plausible values
            if len(ratings) < len(valid_pairs):
                ratings += list(np.random.randint(3, 5, len(valid_pairs) - len(ratings)))
            return ratings

        except Exception as e:
            print("Error in batch:", e)
            return list(np.random.randint(3, 5, len(valid_pairs)))


In [11]:
from surprise import accuracy
from sklearn.model_selection import train_test_split
from surprise import Reader, Dataset



reader = Reader(rating_scale=(1, 5))
data = reviews[["AuthorId", "RecipeId", "Rating"]]
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

recipes["RecipeId"] = recipes["RecipeId"].astype(int)
train_df["RecipeId"] = train_df["RecipeId"].astype(int)
test_df["RecipeId"] = test_df["RecipeId"].astype(int)

# Build user histories
user_histories = {}
grouped = train_df.groupby("AuthorId")
for uid, df in tqdm(grouped):
    # Keep recipe name + rating
    user_histories[uid] = [
        (recipes.loc[recipes["RecipeId"] == rid, "Name"].values[0], int(r))
        for rid, r in zip(df["RecipeId"], df["Rating"])
        if rid in recipes["RecipeId"].values
    ]



100%|██████████| 230674/230674 [05:13<00:00, 735.80it/s] 


In [31]:
recipes["RecipeId"] = recipes["RecipeId"].astype(int)
train_df["RecipeId"] = train_df["RecipeId"].astype(int)
test_df["RecipeId"] = test_df["RecipeId"].astype(int)

# Take a smaller sample of the test set
sample_size = 10000  # adjust depending on patience/budget
sampled_test = test_df.sample(n=sample_size, random_state=4).reset_index(drop=True)

# Initialize model
llm_model = LLMPredictor(recipes, user_histories, batch_size=10, model_path="./tinyllama-merged2")

predictions = []
rows = sampled_test.to_dict(orient="records")

for i in tqdm(range(0, len(rows), llm_model.batch_size)):
    batch = rows[i:i + llm_model.batch_size]
    pairs = [(r["AuthorId"], r["RecipeId"], r["Rating"]) for r in batch]
    ests = llm_model.predict_batch(pairs, )
    for (uid, iid, true_r), est in zip(pairs, ests):
        #print("Estimate for user", uid, "and recipe", iid, "is", est, "true rating is", true_r)
        predictions.append((uid, iid, true_r, est, None))


Loading model from ./tinyllama-merged2...
Model loaded and ready.


100%|██████████| 1000/1000 [23:21<00:00,  1.40s/it]


In [22]:
recipes["RecipeId"] = recipes["RecipeId"].astype(int)
train_df["RecipeId"] = train_df["RecipeId"].astype(int)
test_df["RecipeId"] = test_df["RecipeId"].astype(int)

# Take a smaller sample of the test set
sample_size = 10000  # adjust depending on patience/budget
sampled_test = test_df.sample(n=sample_size, random_state=42).reset_index(drop=True)

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Initialize model
llm_model = LLMPredictor(recipes, user_histories, batch_size=10, model_path=model_name)

predictions_base = []
rows = sampled_test.to_dict(orient="records")

for i in tqdm(range(0, len(rows), llm_model.batch_size)):
    batch = rows[i:i + llm_model.batch_size]
    pairs = [(r["AuthorId"], r["RecipeId"], r["Rating"]) for r in batch]
    ests = llm_model.predict_batch(pairs, )
    for (uid, iid, true_r), est in zip(pairs, ests):
        #print("Estimate for user", uid, "and recipe", iid, "is", est, "true rating is", true_r)
        predictions_base.append((uid, iid, true_r, est, None))


Loading model from TinyLlama/TinyLlama-1.1B-Chat-v1.0...
Model loaded and ready.


100%|██████████| 1000/1000 [45:43<00:00,  2.74s/it] 


In [28]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("./tinyllama-finetuned2")
print(sum(p.numel() for p in model.parameters()))
base = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
print(sum(p.numel() for p in base.parameters()))


1101174784
1100048384


In [26]:
from transformers import AutoModelForCausalLM
m = AutoModelForCausalLM.from_pretrained("./tinyllama-merged2")
print("Merged params:", sum(p.numel() for p in m.parameters()))


Merged params: 1100048384


In [30]:
# check files/sizes in adapter folder
import os
for p in ["./tinyllama-finetuned2", "./tinyllama-merged2"]:
    print("===", p, "===")
    if not os.path.isdir(p):
        print("MISSING", p); continue
    for f in sorted(os.listdir(p)):
        fp = os.path.join(p, f)
        if os.path.isfile(fp):
            print(f, round(os.path.getsize(fp)/1e6,2), "MB")


=== ./tinyllama-finetuned2 ===
README.md 0.01 MB
adapter_config.json 0.0 MB
adapter_model.safetensors 4.52 MB
chat_template.jinja 0.0 MB
special_tokens_map.json 0.0 MB
tokenizer.json 3.62 MB
tokenizer_config.json 0.0 MB
=== ./tinyllama-merged2 ===
chat_template.jinja 0.0 MB
config.json 0.0 MB
generation_config.json 0.0 MB
model.safetensors 4400.22 MB
special_tokens_map.json 0.0 MB
tokenizer.json 3.62 MB
tokenizer_config.json 0.0 MB


In [32]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def compute_mae(predictions):
    """Compute Mean Absolute Error."""
    true = [t for _, _, t, _, _ in predictions]
    pred = [p for _, _, _, p, _ in predictions]
    return mean_absolute_error(true, pred)

def compute_rmse(predictions):
    """Compute Root Mean Squared Error."""
    true = [t for _, _, t, _, _ in predictions]
    pred = [p for _, _, _, p, _ in predictions]
    return np.sqrt(mean_squared_error(true, pred))

def compute_metrics(predictions):
    mae = compute_mae(predictions)
    rmse = compute_rmse(predictions)
    print(f"MAE: {mae:.3f}")
    print(f"RMSE: {rmse:.3f}")
    return mae, rmse

mae, rmse = compute_metrics(predictions)
print("Base model:")
compute_metrics(predictions_base)


MAE: 1.506
RMSE: 2.237
Base model:
MAE: 1.493
RMSE: 2.231


(1.4934, 2.231367293835777)

In [30]:
from collections import defaultdict

def get_top_n(predictions, n=10, min_rating=1, max_rating=5):
    """
    Devuelve las N-mejores recomendaciones para cada usuario de un set de predicciones.

    Args:
        predictions (list[tuple]): Lista de tuplas (uid, iid, true_r, est, _)
        n (int): Número de recomendaciones por usuario
        min_rating (float): Valor mínimo de rating permitido
        max_rating (float): Valor máximo de rating permitido

    Returns:
        dict: {uid: [(iid, est), ...]} con las N mejores recomendaciones por usuario
    """
    top_n = defaultdict(list)

    # Asocia todas las predicciones al usuario correspondiente
    for uid, iid, true_r, est, _ in predictions:
        # Ignorar predicciones inválidas o nulas
        if est is None:
            continue
        # Asegurar que el rating esté en rango válido
        est = float(est)
        est = min(max(est, min_rating), max_rating)
        top_n[uid].append((iid, est))

    # Ordenar y recortar las mejores N recomendaciones
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return dict(top_n)


In [34]:
test_user_items = defaultdict(list)
for uid, iid, r in zip(test_df["AuthorId"], test_df["RecipeId"], test_df["Rating"]):
    if r >= 4:  # only consider ratings 4 or 5 as relevant
        test_user_items[uid].append(iid)


In [35]:
top_n = get_top_n(predictions, n=10)
precision, recall = precision_recall_at_k(top_n, test_user_items, k=10)
ndcg = ndcg_at_k(top_n, test_user_items, k=10)

print(f"Precision@10: {precision:.3f}")
print(f"Recall@10:    {recall:.3f}")
print(f"NDCG@10:      {ndcg:.3f}")

Precision@10: 0.144
Recall@10:    0.417
NDCG@10:      0.559


In [36]:
top_n = get_top_n(predictions_base, n=10)
precision, recall = precision_recall_at_k(top_n, test_user_items, k=10)
ndcg = ndcg_at_k(top_n, test_user_items, k=10)

print(f"Precision@10: {precision:.3f}")
print(f"Recall@10:    {recall:.3f}")
print(f"NDCG@10:      {ndcg:.3f}")

Precision@10: 0.143
Recall@10:    0.422
NDCG@10:      0.561


In [35]:
# Map NutriScore letters to numeric values if needed, e.g. A=5, B=4, ..., E=1
nutri_map = {"A": 5, "B": 4, "C": 3, "D": 2, "E": 1}

def mean_health_at_k(top_n, recipes_df, k=10):
    """
    Computes the mean health (NutriScore) for the top-k recommendations per user.
    
    Args:
        top_n: dict of user_id -> [(recipe_id, predicted_rating), ...]
        recipes_df: dataframe with columns ['RecipeId', 'NutriScore']
        k: top-k items to consider per user
        
    Returns:
        mean_health: float, mean NutriScore (numeric) of top-k items
    """
    all_scores = []
    for uid, recs in top_n.items():
        for iid, _ in recs[:k]:
            row = recipes_df.loc[recipes_df["RecipeId"] == iid]
            if row.empty:
                continue
            score_letter = row.iloc[0]["NutriScore"]
            score = nutri_map.get(score_letter, np.nan)
            if not np.isnan(score):
                all_scores.append(score)
    
    return np.mean(all_scores) if all_scores else np.nan

# Example usage:
# top_n = get_top_n(predictions, n=10)
mean_health = mean_health_at_k(top_n, recipes, k=5)
print(f"MeanHealth@10: {mean_health:.4f}")

MeanHealth@10: 2.8057
